In [1]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
with open("Zayn_Lyrics.txt","r") as f:
    data=f.read()
    data=data.replace('\n','')
    data = data.lower()

In [3]:
data[:50]

"now i'm on the edgei can't find my wayit's inside "

In [4]:
all_chars=list(set(data))
unique_chars = len(all_chars)
total_chars =len(data)

print(unique_chars)
print(total_chars)

41
14662


In [5]:
char_to_ix = { ch:i for i,ch in enumerate(all_chars) }
ix_to_char = { i:ch for i,ch in enumerate(all_chars) }


In [6]:
def generate_batch(seq_length,i):
    inputs = [char_to_ix[ch] for ch in data[i:i+seq_length]]
    targets = [char_to_ix[ch] for ch in data[i+1:i+seq_length+1]]
    inputs=np.array(inputs).reshape(seq_length,1)
    targets=np.array(targets).reshape(seq_length,1)
    return inputs,targets

In [7]:
seq_length = 25
learning_rate = 0.1
num_nodes = 256

In [8]:
def build_rnn(x):
    cell= tf.contrib.rnn.BasicLSTMCell(num_units=num_nodes)
    outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
    print('outputs lpm', outputs.shape)
    return outputs, states

In [9]:
X=tf.placeholder(tf.float32,[None,1])
Y=tf.placeholder(tf.float32,[None,1])


In [10]:
X=tf.cast(X,tf.int32)
Y=tf.cast(Y,tf.int32)

In [11]:
X_onehot=tf.one_hot(X,unique_chars)
Y_onehot=tf.one_hot(Y,unique_chars)

X_onehot

<tf.Tensor 'one_hot:0' shape=(?, 1, 41) dtype=float32>

In [12]:
outputs,states=build_rnn(X_onehot)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
outputs lpm (?, 1, 256)


In [13]:
outputs=tf.transpose(outputs,perm=[1,0,2])

In [14]:
W=tf.Variable(tf.random_normal((num_nodes,unique_chars),stddev=0.001))
B=tf.Variable(tf.zeros((1,unique_chars)))

In [15]:
Ys=tf.matmul(outputs[0],W)+B

In [16]:
prediction = tf.nn.softmax(Ys)

In [17]:
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_onehot,logits=Ys))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [18]:
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)


In [19]:
def predict(seed,i):
    x=np.zeros((1,1))
    x[0][0]= seed
    indices=[]
    for t in range(i):
        p=sess.run(prediction,{X:x})
        index = np.random.choice(range(unique_chars), p=p.ravel())
        x[0][0]=index
        indices.append(index)
    return indices

In [20]:
batch_size=64
total_batch=int(total_chars//batch_size)
epochs=1024
shift=0

In [21]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        print("Epoch {}:".format(epoch))
        if shift + batch_size+1 >= len(data):
            shift =0
        ## get the input and target for each batch by generate_batch
        #function which shifts the input by shift value
        ## and form target
        for i in range(total_batch):
            inputs,targets=generate_batch(batch_size,shift)
            shift += batch_size
            # calculate loss
            if(i%100==0):
                loss=sess.run(cross_entropy,feed_dict={X:inputs,Y:targets})
                
                # We get index of next predicted character by
                # the predict function
                index=predict(inputs[0], 20)
                # pass the index to our ix_to_char dictionary and
                #get the char
                txt = ''.join(ix_to_char[ix] for ix in index)
                print('Iteration %i: '%(i))
                print ('\n %s \n' % (txt, ))
            sess.run(optimiser,feed_dict={X:inputs,Y:targets})

Epoch 0:
Iteration 0: 

 i'q'zxx82[8hbkx)lh): 

Iteration 100: 

 tz?"n:bsj&h?j ok,un& 

Iteration 200: 

 qr y v &ttl a bn tlr 

Epoch 1:
Iteration 0: 

 ' tgunazw, ozu",:8e, 

Iteration 100: 

 iq t& heesqn iqma[ i 

Iteration 200: 

 )ajem e[  iennee usc 

Epoch 2:
Iteration 0: 

 kt.e fl: g8 y?ti,ar  

Iteration 100: 

  oss-p8eit tb d y -& 

Iteration 200: 

 x d: am"(dtileb  ?a  

Epoch 3:
Iteration 0: 

 ii? wwnbeopov )ohqa  

Iteration 100: 

 kia et  e hg iaate a 

Iteration 200: 

 tpb&tsyobs]schahi&t" 

Epoch 4:
Iteration 0: 

 - ,ta&l,hadyat t   h 

Iteration 100: 

 aa y a,dbrflwvg atsw 

Iteration 200: 

 ijior'ysetyiaiaz ost 

Epoch 5:
Iteration 0: 

 fita eohnt  'i tewnh 

Iteration 100: 

 tdw irgyt d  o(yaw(d 

Iteration 200: 

  eflbnefotttotm e rh 

Epoch 6:
Iteration 0: 

 honthieatnspryazttrt 

Iteration 100: 

 emlit' w,y)uornnrjwe 

Iteration 200: 

  w g dttni idziuappa 

Epoch 7:
Iteration 0: 

 letiyosb truogoe yst 

Iteration 100: 

 wetoonn  ee nwe woi: 

I

Iteration 200: 

  pi: llfoon'iyoke, n 

Epoch 63:
Iteration 0: 

 ng heceine ntd ghea  

Iteration 100: 

 t ivbyo'i abonfoughe 

Iteration 200: 

 hi'ss yofleson t wyo 

Epoch 64:
Iteration 0: 

 olof ckut bupig thes 

Iteration 100: 

 a me'sri, thr, it db 

Iteration 200: 

 hed it ou, tinofolin 

Epoch 65:
Iteration 0: 

 tg ivyime me misonit 

Iteration 100: 

 itove md mereshe fan 

Iteration 200: 

  bply, liee se jfehe 

Epoch 66:
Iteration 0: 

 'retoofu dy itandohe 

Iteration 100: 

 werrei'thed s, wsoud 

Iteration 200: 

 rrdsinedl wang and r 

Epoch 67:
Iteration 0: 

 ng boft'sngt mall d  

Iteration 100: 

 f tjft yorimus you f 

Iteration 200: 

 hedo's dher y u abol 

Epoch 68:
Iteration 0: 

 e tnging wane c ware 

Iteration 100: 

 hr ln piedinnennde n 

Iteration 200: 

  r i tesrve berno in 

Epoch 69:
Iteration 0: 

 'yollig ayo mit way  

Iteration 100: 

 the f we wave so d s 

Iteration 200: 

 hasre ndoot'tsauwana 

Epoch 70:
Iteration 0: 

 medacag, id pi y

Iteration 100: 

 y the heeathe wou ar 

Iteration 200: 

  t't l d n of, pzuss 

Epoch 126:
Iteration 0: 

 ayou o the w a r ai  

Iteration 100: 

 se titan wp ys fthan 

Iteration 200: 

 oug tharo s i ghe i' 

Epoch 127:
Iteration 0: 

 thi, i's llli'miniga 

Iteration 100: 

 tike wadyou o jul--c 

Iteration 200: 

 't i'taw atonou my p 

Epoch 128:
Iteration 0: 

 in'jut ana is se fam 

Iteration 100: 

 win'tat whyoue2enher 

Iteration 200: 

  litri chs okenof th 

Epoch 129:
Iteration 0: 

 and y, t'dome, t wr  

Iteration 100: 

 y bontothe ysou'shea 

Iteration 200: 

  bkodhayotht tikerio 

Epoch 130:
Iteration 0: 

 e l tht.naus id ts i 

Iteration 100: 

 in'me deayovts tike  

Iteration 200: 

 isseighe, cace y, fh 

Epoch 131:
Iteration 0: 

  yd, t alktist warie 

Iteration 100: 

 the she yngeaysrdoui 

Iteration 200: 

 he t lon'tar it yod  

Epoch 132:
Iteration 0: 

 't n ipke t tonheave 

Iteration 100: 

 nanofbasen wbe w ili 

Iteration 200: 

  me tant al wacht

Iteration 200: 

 'sen't'tathit've sfe 

Epoch 188:
Iteration 0: 

 n't of i've yinn ig  

Iteration 100: 

 waditauththi stind t 

Iteration 200: 

 rstinandsatin sh she 

Epoch 189:
Iteration 0: 

 ganhthau jugou tsiga 

Iteration 100: 

 thikie d s zolak che 

Iteration 200: 

  aynebuginoou your o 

Epoch 190:
Iteration 0: 

  moft'tiganoffe nd,  

Iteration 100: 

 into w icansousowak  

Iteration 200: 

  cay ke yorwee wo of 

Epoch 191:
Iteration 0: 

 llus an'the, s a ist 

Iteration 100: 

 lalo s s t febove is 

Iteration 200: 

 hethennreri'be zui'm 

Epoch 192:
Iteration 0: 

 g m theso eat, t gha 

Iteration 100: 

 h frthi'toud adoffyo 

Iteration 200: 

 out lou inghe athe t 

Epoch 193:
Iteration 0: 

  couttinat'spu thti  

Iteration 100: 

 ches my, wigwimy waz 

Iteration 200: 

  boolle, le'thedim p 

Epoch 194:
Iteration 0: 

 go ake taus u t i t  

Iteration 100: 

 be donofou're l your 

Iteration 200: 

 owa g lsane yo wrei  

Epoch 195:
Iteration 0: 

 ou lo fr

Iteration 100: 

 atf2si thed blon'vet 

Iteration 200: 

  shathive, t t bowau 

Epoch 251:
Iteration 0: 

 's ju plan i't bupow 

Iteration 100: 

 jusid, yoofats thdo  

Iteration 200: 

 htoused ache me it j 

Epoch 252:
Iteration 0: 

 d wa nthowin litilig 

Iteration 100: 

 waustanghthet s byor 

Iteration 200: 

  ngandnd cans s fay  

Epoch 253:
Iteration 0: 

 'trs istr ie larimef 

Iteration 100: 

 presi wakesheref, de 

Iteration 200: 

  wn sbubyteshebouhed 

Epoch 254:
Iteration 0: 

 t the it youg y, lli 

Iteration 100: 

 lli dyoutin od 'than 

Iteration 200: 

 heackeisabouit i st' 

Epoch 255:
Iteration 0: 

 alurof tofofffou, ig 

Iteration 100: 

 anon t rigf, dour ik 

Iteration 200: 

 heshem l ploit e i i 

Epoch 256:
Iteration 0: 

 ngheeake i'tol lar k 

Iteration 100: 

 us ri jusivebeme]the 

Iteration 200: 

  i, d sa me t o thet 

Epoch 257:
Iteration 0: 

 e it wat pacredoflie 

Iteration 100: 

 toth, by,y, t u fwak 

Iteration 200: 

 rus tcalowefo thi

...

Iteration 100: 

 ploveryoosoupano the 

Iteration 200: 

  out is wht't'rer li 

Epoch 751:
Iteration 0: 

 g cour in fe it bu w 

Iteration 100: 

 you'sangalindarigher 

Iteration 200: 

  int iknd li gime st 

Epoch 752:
Iteration 0: 

 n i'rd dat ksitig y  

Iteration 100: 

 bought w wds hejus w 

Iteration 200: 

 okel g one t t ith s 

Epoch 753:
Iteration 0: 

 'llloousgi'to ind al 

Iteration 100: 

 ive med boou lllde s 

Iteration 200: 

 offiserorimyo indovi 

Epoch 754:
Iteration 0: 

 eesndyolit p r fa ou 

Iteration 100: 

 dnof loures haitande 

Iteration 200: 

 htt lewatsi'tak tary 

Epoch 755:
Iteration 0: 

 t so t dse lita the' 

Iteration 100: 

 t wake y t m warelin 

Iteration 200: 

  yie wanyoungaroue y 

Epoch 756:
Iteration 0: 

 i ingig ttstred w iv 

Iteration 100: 

 y, whtant medari f w 

Iteration 200: 

 out i ayollowarsenou 

Epoch 757:
Iteration 0: 

 olinn iso y ce g tan 

Iteration 100: 

 tteg fi'sshe, cen to 

Iteration 200: 

 't a theeewoved r

Iteration 200: 

 oran'rdcacacedyif ow 

Epoch 813:
Iteration 0: 

  ike wokie y g whtou 

Iteration 100: 

 ba ordourn rs o wana 

Iteration 200: 

  yous ot me tcrd be  

Epoch 814:
Iteration 0: 

 g our fure ll te tht 

Iteration 100: 

 sghe an'tshen'tojou  

Iteration 200: 

  whtitha d in'my, do 

Epoch 815:
Iteration 0: 

  mi'ton, t me tok am 

Iteration 100: 

 rs you caineche t of 

Iteration 200: 

 hanightanfillar th t 

Epoch 816:
Iteration 0: 

  th do t wht'sau m f 

Iteration 100: 

 se i'lyou fimybynga  

Iteration 200: 

  s, youp oloveal ng  

Epoch 817:
Iteration 0: 

 d-ingou tsing routhe 

Iteration 100: 

 renngo ode y llo li  

Iteration 200: 

  wate t, bsthauinn'v 

Epoch 818:
Iteration 0: 

 d in yo we i t'ske t 

Iteration 100: 

 thengh heloforbed mi 

Iteration 200: 

 hame cenn't wake i w 

Epoch 819:
Iteration 0: 

  jutheaniou w hi it  

Iteration 100: 

 l yoush ybonsheak ps 

Iteration 200: 

 in yowalou't g fesha 

Epoch 820:
Iteration 0: 

 ay wowar

Iteration 100: 

 ul ays holi s d ps j 

Iteration 200: 

 alonglpallay winiset 

Epoch 876:
Iteration 0: 

 onged re dlofo w w i 

Iteration 100: 

 'm torinedadathakeam 

Iteration 200: 

  i d fin y whindo t  

Epoch 877:
Iteration 0: 

 wanoigh o wborki wat 

Iteration 100: 

 ingay wheinoore whal 

Iteration 200: 

 he irdoui u' we whe, 

Epoch 878:
Iteration 0: 

 doustighay oke w i w 

Iteration 100: 

 yo we ishayo, itheet 

Iteration 200: 

  l t meangis y, rse  

Epoch 879:
Iteration 0: 

 t inont yndang 'cait 

Iteration 100: 

 sthtthanou, thi s rs 

Iteration 200: 

 harond titi inane r  

Epoch 880:
Iteration 0: 

 gou't awabje, ime wh 

Iteration 100: 

 w lloffouis al t the 

Iteration 200: 

 im wacke, bx]hint ll 

Epoch 881:
Iteration 0: 

  kit tthabout, wher  

Iteration 100: 

 y, yoou'dinde t you  

Iteration 200: 

 hengh t'trinolo amof 

Epoch 882:
Iteration 0: 

 don wst ithatou opai 

Iteration 100: 

 t's rs folous ourdow 

Iteration 200: 

  whe'lpl onteatem

Iteration 200: 

 chinswaconis me ynon 

Epoch 938:
Iteration 0: 

 dse ak inakees yofak 

Iteration 100: 

 t m hake theru jure  

Iteration 200: 

  ily pfll it by, ant 

Epoch 939:
Iteration 0: 

 ghaneroffu ff ti'the 

Iteration 100: 

 yo p y, wo tane uis  

Iteration 200: 

 his on'chare yolandr 

Epoch 940:
Iteration 0: 

 oou se ingst mondyi' 

Iteration 100: 

 s catan y im isshyou 

Iteration 200: 

 i''to raron'tt wabli 

Epoch 941:
Iteration 0: 

 allis i't i ig lithe 

Iteration 100: 

 y yoto do wandht ins 

Iteration 200: 

  to beyofansatet mac 

Epoch 942:
Iteration 0: 

 oveame m jure llle[h 

Iteration 100: 

 tce, titre o jouprou 

Iteration 200: 

 onar ovinangithe so  

Epoch 943:
Iteration 0: 

 allsay, live ineive  

Iteration 100: 

 y fare t t byntoousi 

Iteration 200: 

 cakee th owhar h fon 

Epoch 944:
Iteration 0: 

 ofay at wang byffout 

Iteration 100: 

 nin owan i'thamethee 

Iteration 200: 

 i'taloveanke t gs fe 

Epoch 945:
Iteration 0: 

 t stau w

Iteration 100: 

 cke s whangofon'ckir 

Iteration 200: 

 anyo li:]i't i shei' 

Epoch 1001:
Iteration 0: 

 nat t[u a byoune ane 

Iteration 100: 

 aryoreve y, me d he  

Iteration 200: 

 'to m larabe i chau  

Epoch 1002:
Iteration 0: 

 dintoutsouseit t lke 

Iteration 100: 

 lllyin f wha bofbrea 

Iteration 200: 

 tovinntaui't rer ith 

Epoch 1003:
Iteration 0: 

 'tatal fikistrecan't 

Iteration 100: 

 f, pldeith anout be  

Iteration 200: 

  aree ut ri's cau're 

Epoch 1004:
Iteration 0: 

 galita au, bo me bof 

Iteration 100: 

 whthty wasazhtannd l 

Iteration 200: 

 hig li nginnann nhei 

Epoch 1005:
Iteration 0: 

 'toverst cakerou few 

Iteration 100: 

 y we t n pe les, hev 

Iteration 200: 

 he yobl hee yethase, 

Epoch 1006:
Iteration 0: 

 igs ist'r lit bening 

Iteration 100: 

 dsilonout fri'sit ch 

Iteration 200: 

 he wore st dard ixgh 

Epoch 1007:
Iteration 0: 

 ooupulour meal ake f 

Iteration 100: 

 ke rin byone mea yon 

Iteration 200: 

  pso'sokn'